<a href="https://colab.research.google.com/github/alyssonmazoni/institutions_cognitive/blob/main/Cognitive_distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cognitive and geographical distances of scholarly institutions

Data were collected from a cloud stored version of the OpenAlex full dump. This version is structured to tables as in the code (https://github.com/alyssonmazoni/openalex.git) into the Google BigQuery platform. The tables assembled can be queried for the relevant information such as latitude, longitude, country codes, number of works and number of authors.

The classification of works based on clusters by bibliographic coupling is collected from (https://www.leidenmadtrics.nl/articles/an-open-approach-for-classifying-research-publications, https://zenodo.org/records/10560276). The tables containing the clusters (macro, meso and micro level) are also uploaded to the same environment as a new dataset. That way, it is possible to collect further metadata related to the institutions.

We have worked with a filter of the largest institutions as the ones with more than 60 thousand works indexed in OpenAlex. For every one of these institutions, we have collected the macro cluster identifications for all the works. We match works and institutions using the authorships’ tables in OpenAlex (https://github.com/alyssonmazoni/openalex).

It is possible to convert the table of works and their clusters for the selected institutions into a different format where every row contains an institution and there are columns for every cluster number. This is done by pivoting the table for the clusters and aggregating the works by institutions.

After this, if we normalise the number of works in every cluster of every institution by the number of works in total in the institution, we achieve a vector representation of the research effort that a given institution related to the area classification. Two institutions can be compared by how similar are their cognitive efforts if these vectors are more or less aligned. To that effect we use the scalar product (cosine) of the two vectors representing them. The resulting number is 1 for perfectly aligned institutions and zero for institutions that work on completely disjoint areas.

Using the distance and collaboration metadata available in OpenAlex, it is possible to calculate a geographical distance (by latitude and longitude) and a collaboration distance as the number of works with shared authorship between two given institutions.


# Preparation

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
import numpy as np
from numpy import (array, dot, arccos, clip)
from numpy.linalg import norm

import pandas as pd
import geopy.distance

In [ ]:
%%bigquery --project=insyspo

DROP TABLE IF EXISTS `insyspo.userdb_alysson_mazoni.institutions_clusters`;
CREATE TABLE `insyspo.userdb_alysson_mazoni.institutions_clusters` AS (
    SELECT
      CAST(SPLIT(authorships.institution_id,'/I')[1] AS INT) AS institution_id,
      clusters.macro_cluster_id,
      COUNT(DISTINCT(works.id)) AS nw, # distinct works from institution in the cluster
      SUM(1/works.authors_count) AS nc # authors' fractional contribution in all works of inst and cluster
    FROM `insyspo.publicdb_cwts_openalex_areas.clustering` AS clusters
    JOIN `insyspo.publicdb_openalex_2024_02_rm.works` AS works
    ON clusters.doi = SUBSTR(works.doi,17)
    JOIN `insyspo.publicdb_openalex_2024_02_rm.works_authorships` AS authorships
    ON works.id = authorships.work_id
    GROUP BY authorships.institution_id, clusters.macro_cluster_id
);


# Institutions and their works in clusters

In [8]:
%%bigquery df --project=insyspo

SELECT a.institution_id,
  a.macro_cluster_id,
  a.nw/b.works_count AS nw,
  a.nc/b.works_count AS nc,
FROM `insyspo.userdb_alysson_mazoni.institutions_clusters` AS a
JOIN  `insyspo.publicdb_openalex_2024_02_rm.institutions` AS b
ON a.institution_id = CAST(SPLIT(b.id,'/I')[1] AS INT)
WHERE b.works_count > 0 AND b.works_count > 60000
ORDER BY a.institution_id

Query is running:   0%|          |

Downloading:   0%|          |

In [9]:
df.head()

,institution_id,macro_cluster_id,nw,nc
0,241749,18,0.012287,0.008505
1,241749,14,0.019671,0.007482
2,241749,2,0.088461,0.053241
3,241749,13,0.012386,0.006066
4,241749,20,0.004208,0.002457


In [ ]:
%%bigquery  --project=insyspo

DROP TABLE IF EXISTS `insyspo.userdb_alysson_mazoni.institutions_100000`;
CREATE TABLE `insyspo.userdb_alysson_mazoni.institutions_100000` AS (
  SELECT DISTINCT a.institution_id,
    b.display_name,
    b.latitude,
    b.longitude,
    b.works_count,
  FROM `insyspo.userdb_alysson_mazoni.institutions_clusters` AS a
  JOIN  `insyspo.publicdb_openalex_2024_02_rm.institutions` AS b
  ON a.institution_id = CAST(SPLIT(b.id,'/I')[1] AS INT)
  WHERE b.works_count > 0 AND b.works_count > 100000
  ORDER BY a.institution_id
)

Query is running:   0%|          |

""


In [7]:
%%bigquery  --project=insyspo

DROP TABLE IF EXISTS `insyspo.userdb_alysson_mazoni.institutions_60000`;
CREATE TABLE `insyspo.userdb_alysson_mazoni.institutions_60000` AS (
  SELECT DISTINCT a.institution_id,
    b.display_name,
    b.latitude,
    b.longitude,
    b.works_count,
  FROM `insyspo.userdb_alysson_mazoni.institutions_clusters` AS a
  JOIN  `insyspo.publicdb_openalex_2024_02_rm.institutions` AS b
  ON a.institution_id = CAST(SPLIT(b.id,'/I')[1] AS INT)
  WHERE b.works_count > 0 AND b.works_count > 60000
  ORDER BY a.institution_id
)

Query is running:   0%|          |

""


In [10]:
df.head()

,institution_id,macro_cluster_id,nw,nc
0,241749,18,0.012287,0.008505
1,241749,14,0.019671,0.007482
2,241749,2,0.088461,0.053241
3,241749,13,0.012386,0.006066
4,241749,20,0.004208,0.002457


## Pivot to get clusters as columns

In [11]:
df = df.pivot(index='institution_id', columns='macro_cluster_id', values=['nw','nc']).fillna(0)
df = df.sort_values(by='institution_id')
df.head()

nw                                                    \
macro_cluster_id        1         2         3         4         5         6    
institution_id                                                                 
241749            0.118760  0.088461  0.078101  0.043849  0.028165  0.032361   
848706            0.242074  0.196250  0.063912  0.033158  0.046162  0.025544   
1174212           0.140423  0.033321  0.100453  0.016193  0.053318  0.031545   
1850255           0.156383  0.078907  0.100133  0.053580  0.012031  0.026358   
2613432           0.129279  0.050615  0.139617  0.025762  0.036445  0.029813   

                                                          ...        nc  \
macro_cluster_id        7         8         9         10  ...        11   
institution_id                                            ...             
241749            0.038611  0.030066  0.013661  0.014888  ...  0.010027   
848706            0.016516  0.005010  0.052385  0.005400  ...  0.015949   
1174212           0.028979  0.016636  0.035614  0.013804  ...  0.006109   
1850255           0.039731  0.015101  0.004887  0.008343  ...  0.001546   
2613432           0.026910  0.017982  0.025365  0.029626  ...  0.003360   

                                                                              \
macro_cluster_id        12        13        14        15        16        17   
institution_id                                                                 
241749            0.008862  0.006066  0.007482  0.004454  0.004224  0.016551   
848706            0.007437  0.026591  0.013829  0.020683  0.011692  0.005285   
1174212           0.007187  0.016790  0.007404  0.013046  0.002019  0.003519   
1850255           0.007286  0.009637  0.024211  0.013998  0.000775  0.008714   
2613432           0.008501  0.013790  0.012054  0.009446  0.004840  0.001407   

                                                
macro_cluster_id        18        19        20  
institution_id                                  
241749            0.008505  0.000320  0.002457  
848706            0.003308  0.000042  0.001445  
1174212           0.002241  0.000235  0.005447  
1850255           0.000671  0.000145  0.001074  
2613432           0.001040  0.000268  0.001727  

[5 rows x 40 columns]

In [12]:
df.shape

(474, 40)

# Cognitive distance

In [13]:
institutions = list(df.index)
insta = []
instb = []
cd = []
for i,inst1 in enumerate(institutions):
  print(i,inst1)
  for j,inst2 in enumerate(institutions):
    if inst2>inst1:
      a = df.loc[inst1,'nw'].to_numpy()
      b = df.loc[inst2,'nw'].to_numpy()
      c = dot(a,b)/norm(a)/norm(b)
      insta += [inst1]
      instb += [inst2]
      cd += [c]

Tcd = pd.DataFrame({'inst1':insta,
                    'inst2':instb,
                    'cognitive':cd})

0 241749
1 848706
2 1174212
3 1850255
4 2613432
5 3923682
6 4068193
7 4104125
8 4432739
9 4575257
10 4577782
11 4654613
12 5023651
13 5124864
14 5388228
15 5681781
16 7877124
17 7882870
18 8087733
19 8204097
20 8692664
21 8961855
22 9224756
23 9360294
24 9617848
25 9842412
26 11701301
27 12097938
28 12315562
29 13469542
30 14243506
31 15057530
32 16097986
33 16391192
34 16609230
35 16733864
36 17145004
37 17974374
38 18014758
39 19820366
40 19880235
41 19894307
42 20089843
43 20231570
44 21250087
45 21370196
46 21491767
47 22299242
48 22465464
49 23923803
50 23946033
51 24185976
52 24354313
53 24603500
54 24943067
55 25217355
56 25254941
57 25757504
58 25974101
59 26092322
60 27357992
61 27804330
62 27837315
63 28022161
64 28166907
65 28290843
66 28407311
67 31512782
68 31746571
69 32021983
70 32389192
71 32597200
72 32625721
73 32971472
74 33213144
75 33849332
76 34352273
77 34809795
78 35440088
79 36234482
80 36243813
81 36258959
82 37461747
83 38877650
84 39343248
85 39422238
86 395

In [14]:
Tcd.dtypes

inst1          int64
inst2          int64
cognitive    float64
dtype: object

In [49]:
Tcd.to_gbq(project_id = 'insyspo',
        destination_table = 'userdb_alysson_mazoni.institutions_cognitive_distance',
        if_exists = 'replace')

100%|██████████| 1/1 [00:00<00:00, 1580.37it/s]


# Geographical distance

In [51]:
%%bigquery df --project=insyspo

SELECT DISTINCT a.institution_id,
  b.latitude,
  b.longitude,
  b.country_code
FROM `insyspo.userdb_alysson_mazoni.institutions_clusters` AS a
JOIN  `insyspo.publicdb_openalex_2024_02_rm.institutions` AS b
ON a.institution_id = CAST(SPLIT(b.id,'/I')[1] AS INT)
WHERE b.works_count > 0  AND b.works_count > 60000

Query is running:   0%|          |

Downloading:   0%|          |

In [52]:
df = df.sort_values(by='institution_id')
institutions = list(df['institution_id'])
insta = []
instb = []
gd = []

for i,inst1 in enumerate(institutions):
  print(i,inst1)
  for j,inst2 in enumerate(institutions):
    if inst2>inst1:
      a = df.loc[df['institution_id']==inst1,['latitude','longitude']].to_numpy()
      b = df.loc[df['institution_id']==inst2,['latitude','longitude']].to_numpy()
      c = geopy.distance.geodesic(a, b).km
      insta += [inst1]
      instb += [inst2]
      gd += [c]

Tgd = pd.DataFrame({'inst1':insta,'inst2':instb,
                    'geographical':gd})

0 241749
1 848706
2 1174212
3 1850255
4 2613432
5 3923682
6 4068193
7 4104125
8 4432739
9 4575257
10 4577782
11 4654613
12 5023651
13 5124864
14 5388228
15 5681781
16 7877124
17 7882870
18 8087733
19 8204097
20 8692664
21 8961855
22 9224756
23 9360294
24 9617848
25 9842412
26 11701301
27 12097938
28 12315562
29 13469542
30 14243506
31 15057530
32 16097986
33 16391192
34 16609230
35 16733864
36 17145004
37 17974374
38 18014758
39 19820366
40 19880235
41 19894307
42 20089843
43 20231570
44 21250087
45 21370196
46 21491767
47 22299242
48 22465464
49 23923803
50 23946033
51 24185976
52 24354313
53 24603500
54 24943067
55 25217355
56 25254941
57 25757504
58 25974101
59 26092322
60 27357992
61 27804330
62 27837315
63 28022161
64 28166907
65 28290843
66 28407311
67 31512782
68 31746571
69 32021983
70 32389192
71 32597200
72 32625721
73 32971472
74 33213144
75 33849332
76 34352273
77 34809795
78 35440088
79 36234482
80 36243813
81 36258959
82 37461747
83 38877650
84 39343248
85 39422238
86 395

In [53]:
df.head()

,institution_id,latitude,longitude,country_code
164,241749,52.200000,0.116670,GB
89,848706,37.588223,126.993610,KR
394,1174212,34.052230,-118.243680,US
176,1850255,47.558390,7.573270,CH
307,2613432,28.054562,-82.413055,US


In [54]:
Tgd.to_gbq(project_id = 'insyspo',
        destination_table = 'userdb_alysson_mazoni.institutions_geographical_distance',
        if_exists = 'replace')

100%|██████████| 1/1 [00:00<00:00, 764.55it/s]


# Collaboration distance

In [ ]:
%%bigquery df --project=insyspo

DROP TABLE IF EXISTS `insyspo.userdb_alysson_mazoni.institutions_collaborations`;
CREATE TABLE `insyspo.userdb_alysson_mazoni.institutions_collaborations` AS (
  SELECT authorships1.institution_id AS inst1,
    authorships2.institution_id AS inst2,
    COUNT(DISTINCT(works.id)) AS works
  FROM `insyspo.publicdb_cwts_openalex_areas.clustering` AS clusters
  JOIN `insyspo.publicdb_openalex_2024_02_rm.works` AS works
  ON clusters.doi = SUBSTR(works.doi,17)
  JOIN `insyspo.publicdb_openalex_2024_02_rm.works_authorships` AS authorships1
  ON works.id = authorships1.work_id
  JOIN `insyspo.publicdb_openalex_2024_02_rm.works_authorships` AS authorships2
  ON works.id = authorships2.work_id
  WHERE authorships1.institution_id > authorships2.institution_id
  GROUP BY  authorships1.institution_id,
    authorships2.institution_id
  ORDER BY authorships1.institution_id, authorships2.institution_id
)

Query is running:   0%|          |

In [4]:
%%bigquery Tid --project=insyspo

DROP TABLE IF EXISTS `insyspo.userdb_alysson_mazoni.institutions_collaboration_distance`;
CREATE TABLE `insyspo.userdb_alysson_mazoni.institutions_collaboration_distance` AS (
  SELECT CAST(SPLIT(a.inst1,'/I')[1] AS INT) AS inst1,
    CAST(SPLIT(a.inst2,'/I')[1] AS INT) AS inst2,
    works/SQRT(b.works_count*c.works_count) AS collaboration,
    works AS w_collaboration,
    b.works_count AS inst1_works_count,
    c.works_count AS inst2_works_count,
  FROM `insyspo.userdb_alysson_mazoni.institutions_collaborations` AS a
  JOIN  `insyspo.publicdb_openalex_2024_02_rm.institutions` AS b
  ON a.inst1 = b.id
  JOIN  `insyspo.publicdb_openalex_2024_02_rm.institutions` AS c
  ON a.inst2 = c.id
  WHERE b.works_count > 60000 AND c.works_count > 60000
  ORDER BY CAST(SPLIT(a.inst1,'/I')[1] AS INT),  CAST(SPLIT(a.inst2,'/I')[1] AS INT)
)

Query is running:   0%|          |

# Join tables with pairs of institutions

In [55]:
%%bigquery --project=insyspo

DROP TABLE IF EXISTS `insyspo.userdb_alysson_mazoni.institutions_all_distances`;
CREATE TABLE `insyspo.userdb_alysson_mazoni.institutions_all_distances` AS (
  SELECT a.inst1, a.inst2,
    a.cognitive,
    b.geographical,
    c.collaboration,
    c.w_collaboration,
    c.inst1_works_count,
    c.inst2_works_count,
  FROM insyspo.userdb_alysson_mazoni.institutions_cognitive_distance AS a
  LEFT JOIN insyspo.userdb_alysson_mazoni.institutions_geographical_distance AS b
  ON (a.inst1 = b.inst1 AND a.inst2 = b.inst2)
   OR (a.inst1 = b.inst2 AND a.inst2 = b.inst1)
  LEFT JOIN insyspo.userdb_alysson_mazoni.institutions_collaboration_distance AS c
  ON (a.inst1 = c.inst1 AND a.inst2 = c.inst2)
   OR (a.inst1 = c.inst2 AND a.inst2 = c.inst1)
)

Executing query with job ID: d526c7df-3440-42f0-970c-4ee53f951976
Query executing: 50.22s


ERROR:
 400 Query exceeded resource limits. This query used 29804 CPU seconds but would charge only 30M Analysis bytes. This exceeds the ratio supported by the on-demand pricing model. Please consider moving this workload to the flat-rate reservation pricing model, which does not have this limit. 29804 CPU seconds were used, and this query must use less than 7600 CPU seconds. at [2:1]

Location: US
Job ID: d526c7df-3440-42f0-970c-4ee53f951976



# Tests

In [ ]:
%%bigquery --project=insyspo


DROP TABLE IF EXISTS `insyspo.userdb_alysson_mazoni.institutions`;
CREATE TABLE `insyspo.userdb_alysson_mazoni.institutions` AS (
  SELECT b.inst1 AS inst_id,
    a.*
  FROM insyspo.publicdb_openalex_2024_02_rm.institutions AS a
  JOIN `insyspo.userdb_alysson_mazoni.institutions_all_distances` AS b
  ON  CAST(SPLIT(a.id,'/I')[1] AS INT) = b.inst1
)


Query is running:   0%|          |

""


In [ ]:
%%bigquery --project=insyspo

DROP TABLE IF EXISTS `insyspo.userdb_alysson_mazoni.institutions_final`;
CREATE TABLE `insyspo.userdb_alysson_mazoni.institutions_final` AS (
SELECT DISTINCT b.display_name AS inst1_name,
  c.display_name AS inst2_name,
  b.latitude AS lata,
  b.longitude AS longa,
  c.latitude AS latb,
  c.longitude AS longb,
  a.*
FROM `insyspo.userdb_alysson_mazoni.institutions_all_distances` AS a
JOIN `insyspo.userdb_alysson_mazoni.institutions` AS b
ON a.inst1 = b.inst_id
JOIN `insyspo.userdb_alysson_mazoni.institutions` AS c
ON a.inst2 = c.inst_id
ORDER BY inst1, inst2
)

Query is running:   0%|          |

""
